In [1]:
import pandas as pd
import numpy as np
import torch
import os
import datetime
import math
import matplotlib.pyplot as plt
import random
__current_path__ = "/Users/kyle/Documents/AI534/IA2"
__current_year__ = datetime.date.today().year
global global_training_data
np.random.seed(42)

In [ ]:
def load_data(path):
    # Your code here:
    loaded_data = pd.read_csv(path)
    return loaded_data
def sigmoid(x):
    return (1 /(1+math.exp(-x)))
def preprocess_data(data):
    # Your code here:
    normalize_list = ["Age", "Annual_Premium", "Vintage"]
    for target_column in normalize_list:
        mean = global_training_data[target_column].mean()
        s_d = global_training_data[target_column].std()
        for index, each_row in enumerate(data[target_column]):
                data.loc[index, target_column] = (each_row - mean) / s_d
    preprocessed_data = data
    return preprocessed_data

In [ ]:
def predict_proba(X, weights):
    vsigmoid = np.vectorize(sigmoid)
    return vsigmoid(X.dot(weights.T))
def predict(X, weights, threhold):
    vsigmoid = np.vectorize(sigmoid)
    return np.where((vsigmoid(X.dot(weights.T)))>threhold,1,0)

In [ ]:
def check_sparsity(data, threshold=math.pow(10, -6)):
    if threshold == 0:
        return np.count_nonzero(data == threshold)
    else:
        return np.count_nonzero(abs(data) < threshold)

In [ ]:
def LR_L2_train(train_data, val_data, lambda_):
    data = train_data
    valid = val_data
    device = torch.device('mps')
    vsigmoid = torch.nn.Sigmoid()
    labels = data.iloc[:, -1:]
    data = data.iloc[:, :-1]
    valid_labels = valid.iloc[:, -1:]
    valid = valid.iloc[:, :-1]
    weights = []
    N = len(data)
    N_valid = len(valid)
    for j in range(0,len(data.columns)):
        weights.append(random.uniform(-1, 1))
    weights = torch.tensor([weights],dtype=torch.float32,device=device)
    X = torch.tensor(data.values,dtype=torch.float32,device=device)
    Y = torch.tensor(labels.values,dtype=torch.float32,device=device)
    X_valid = torch.tensor(valid.values,dtype=torch.float32,device=device)
    Y_valid = torch.tensor(valid_labels.values,dtype=torch.float32,device=device)
    # Train Model
    for lr in [-2.709999999999987]:
        lr = math.pow(10,lr)
        # wandb.init(project="IA2_Lambda4", entity="ai543gp55")
        # wandb.run.name = str(lambda_)
        # # wandb.run.name = 'main'
        # cfg = wandb.config
        # cfg.update({"lr" : lr, "l1_size" : 0, "l2_size": lambda_})

        epoch = 0
        acc_train = []
        acc_valid = []
        min_MSE = 1000
        last_weight_update = 0
        while True:
            # Batch Logistic Regression
            gradient_t = Y - vsigmoid(torch.tensordot(X,weights.T,dims=1))
            gradient = torch.mul(torch.tensordot(X.T,gradient_t,dims=1),(lr/N))
            weights += gradient.T
            weights.T[1:] -= torch.mul(weights.T[1:],(lr*lambda_))
            # gradient_t = Y - vsigmoid(X.dot(weights.T))
            # gradient = (X.T.dot(gradient_t)).dot(lr/N)
            # # Loss
            # loss = (- Y.T.dot(np.log(vsigmoid(X.dot(weights.T)))) - (np.subtract(1,Y.T).dot(np.log(np.subtract(1,(vsigmoid(X.dot(weights.T)))))))).dot(1/N) + ((np.sum(np.power(weights.T[1:],2)))*lambda_)
            loss = torch.mul(- torch.tensordot(Y.T,(torch.log(vsigmoid(torch.tensordot(X,weights.T,dims=1)))),dims=1) - torch.tensordot(torch.subtract(1,Y.T),(torch.log(torch.subtract(1,(vsigmoid(torch.tensordot(X,weights.T,dims=1))))))),(1/N)) + ((torch.sum(torch.pow(weights.T[1:],2)))*lambda_)
            # # Converge at Threshold
            # current_gradient = np.linalg.norm(np.subtract((gradient_t.dot(1/N)),(lambda_*weights)))
            current_gradient = torch.norm(torch.subtract((torch.mul(gradient_t,(1/N))),torch.mul(lambda_,weights)))
            if current_gradient < math.pow(10,-4):
                print("Converge with ",min_MSE," At Epoch: ",epoch)
                break
            if epoch > 1000:
                print("Stop Training with ",min_MSE," At Epoch: ",epoch, " With LR: ", math.pow(10,-lr))
                break
            # if current_gradient < min_MSE and current_gradient != 0:
            #     min_MSE = current_gradient
            #     last_weight_update = 0
            #     # print(min_MSE," At Epoch: ",epoch)
            # weights += gradient.T
            # # Apply L2 Norm
            # weights.T[1:] -= weights.T[1:].dot(lr*lambda_)
            
            # # Compute Accuracy For Training
            # correct_pred = (predict(X, weights, 0.5) == Y)
            # acc_train_i = (np.sum(correct_pred)/N)*100
            # # Compute Accuracy For Validation
            # correct_pred = (predict(X_valid, weights, 0.5) == Y_valid)
            # acc_valid_i = (np.sum(correct_pred)/N_valid)*100
            # acc_train.append(acc_train_i)
            # acc_valid.append(acc_valid_i)
            epoch += 1
            last_weight_update += 1
            # wandb.log({"epoch": epoch, "lambda": lambda_,"sparsity": check_sparsity(weights) , "loss": loss, "gradient": current_gradient, "weights": weights, "Training Accuracy": acc_train_i, "Validation Accuracy": acc_valid_i})
        # wandb.finish()
    # return weights, acc_train, acc_valid
    return weights.cpu().numpy()




In [ ]:
training_data = load_data(__current_path__+"/IA2-train.csv")
training_data_noisy = load_data(__current_path__+"/IA2-train-noisy.csv")
validation_data = load_data(__current_path__+"/IA2-dev.csv")
global_training_data = training_data.copy(deep=True)
preprocess_data(training_data)
preprocess_data(training_data_noisy)
preprocess_data(validation_data)

In [ ]:
result = []
for i in np.arange(-12,6,1):
    # weights, acc_train, acc_valid = LR_L2_train(training_data,validation_data,math.pow(10,i))
    weights = LR_L2_train(training_data,validation_data,math.pow(10,i))
    # result.append([weights, acc_train, acc_valid,math.pow(10,i)])
    result.append([weights,math.pow(10,i)])